## END-TO-END DEEP LEARNING CONSTRUCTION

#### Outline: Compare runs, choose a model, and deploy it to a REST API

In this starter, I will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature


In [3]:
data=pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv', sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=1234)


In [5]:
# for train set
train_x=train.drop(['quality'], axis=1).values
train_y=train[['quality']].values.ravel()

# for test test
test_x=test.drop(['quality'], axis=1).values
test_y=test[['quality']].values.ravel()

# for validation set
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.2,random_state=1234)

# infer signature
signature = infer_signature(train_x, train_y)

In [6]:
## ANN model

def train_model(params,epochs,
                train_x, train_y, 
                valid_x, valid_y,
                test_x, test_y):
    ## Define model architecture
    mean = np.mean(train_x, axis=0) # columnwise
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## Log the model
        mlflow.tensorflow.log_model(model,"model", signature=signature)

        return {"loss":eval_rmse, "status":STATUS_OK, "model":model}



In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(params, epochs=3,train_x=train_x,train_y=train_y,valid_x=valid_x,valid_y=valid_y,test_x=test_x,test_y=test_y)
    
    return result

In [8]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [9]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct hyperparameter seach using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/01/13 21:04:02 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 17s 398ms/step - loss: 33.4308 - root_mean_squared_error: 5.7819
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 22.3307 - root_mean_squared_error: 4.6998 - val_loss: 6.4508 - val_root_mean_squared_error: 2.5398

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.1595 - root_mean_squared_error: 2.4818
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8054 - root_mean_squared_error: 2.1877 - val_loss: 2.4157 - val_root_mean_squared_error: 1.5542

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.5257 - root_mean_squared_error: 1.5892
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3489 - root_mean_squared_error: 1.5324 - val_loss: 1.8759 - val_root_mean_squared_error: 1.3697

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.1707 - root_mean_squared_error: 1.4733
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [6]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/7284a938f051422ebbf8d5e3900815f6/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/Users/ameetran/Mlflowstarter/mlflowproject/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step


array([[5.562601 ],
       [6.048821 ],
       [5.842861 ],
       ...,
       [5.9265904],
       [5.361891 ],
       [6.2966433]], dtype=float32)

In [7]:
# Load model as a PyFuncModel
model_uri = 'runs:/7284a938f051422ebbf8d5e3900815f6/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas dataframe
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step


array([[5.562601 ],
       [6.048821 ],
       [5.842861 ],
       ...,
       [5.9265904],
       [5.361891 ],
       [6.2966433]], dtype=float32)

In [8]:
# Register best model
mlflow.register_model(model_uri=model_uri,name='best-wine-quality')

Successfully registered model 'best-wine-quality'.
Created version '1' of model 'best-wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1736799294348, current_stage='None', description=None, last_updated_timestamp=1736799294348, name='best-wine-quality', run_id='7284a938f051422ebbf8d5e3900815f6', run_link=None, source='file:///Users/ameetran/Mlflowstarter/Deep-Learning-Model-with-MLflow/mlruns/382913742762027384/7284a938f051422ebbf8d5e3900815f6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>